In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
%matplotlib inline

# Data files full clean

In [2]:
import os
import re

if not os.path.exists("./cleandata"):
    os.makedirs("./cleandata")
content = os.listdir("./newdata")

for item in content:
    # delete the text files and the urls files to clean the library
    #print(item)
    if item.find(".txt") > 0:
        # print(item)
        with open("./newdata/{}".format(item), "r", encoding="iso-8859-1") as f:
            lines = f.readlines()
        # if (os.path.exists("./cleandata/{}".format(item))):
        with open("./cleandata/{}".format(item), "w+") as f:

            for line in lines:
                if "10 Messungen" in line.strip("\n"):
                    line = line[line.find("Ch") :]
                    # print(newline)
                # if (line.strip("\n") != "þStartfischer1 p33-6 hf8 fc6x 0.9km")
                if "Ch" in line.strip("\n") or "V" in line.strip("\n"):
                    f.write(line.replace(";", ""))
f.close()

# print(content)

# Extract the Data and convert it to Dataframes and Excel files

In [56]:
import xlwt
a={}
lst={}
lstdeltas={}
lstverhalt={}
content = os.listdir("./cleandata")
result=pd.DataFrame()
verhaeltnis=pd.DataFrame()
deltas=pd.DataFrame()
for x in content:
    if (x.find(".txt") > 0 and x != ".git"):
        #print(x)
        df = pd.read_csv("./cleandata/{}".format(x),names=["channal", "min", "max", "x", "y"],sep="\s+",encoding="ISO-8859-1")
        dx = df.drop(["x", "y"], axis=1)
        dx["min"] = dx["min"]  # .map(lambda x: x.lstrip("+-;").rstrip("aAbBcC;"))
        dx["channal"] = dx["channal"]  # .map(lambda x: x.lstrip(";").rstrip(";"))
        if 100 in dx.index:
            dx = dx.drop(100)
        dx["min"] = pd.to_numeric(dx["min"])
        dx["max"] = pd.to_numeric(dx["max"])
        dx["deltas"] = dx["max"] - dx["min"]
        #dx["name"]=x.strip('.txt')
        ########### Extract channals ######

        ch0=dx[dx.channal == "Ch0:"]
        ch1=dx[dx.channal == "Ch1:"]
        ch2=dx[dx.channal == "Ch2:"]
        ch3=dx[dx.channal == "Ch3:"]
        ch4=dx[dx.channal == "Ch4:"]
        ch5=dx[dx.channal == "Ch5:"]
        ch6=dx[dx.channal == "Ch6:"]
        ch7=dx[dx.channal == "Ch7:"]
        ch8=dx[dx.channal == "Ch8:"]
        #print(ch0.head())
        ############ build the deltas #############
        delta0= ch0["max"] -  ch0["min"]
        lst['delta0']=delta0
        delta1= ch1["max"] -  ch1["min"]
        lst['delta1']=delta1
        delta2= ch2["max"] -  ch2["min"]
        lst['delta2']=delta2
        delta3= ch3["max"] -  ch3["min"]
        lst['delta3']=delta3
        delta4= ch4["max"] -  ch4["min"]
        lst['delta4']=delta4
        delta5= ch5["max"] -  ch5["min"]
        lst['delta5']=delta5
        delta6= ch6["max"] -  ch6["min"]
        lst['delta6']=delta6
        delta7= ch7["max"] -  ch7["min"]
        lst['delta7']=delta7
        delta8= ch8["max"] -  ch8["min"]
        lst['delta8']=delta8
        #print(delta0.head())
        ##################### format reset index

        df_new = pd.DataFrame.from_dict(lst)
        df_new=df_new.apply(lambda x: pd.Series(x.dropna().values))
        #print(df_new.head(3))
        #df_new.to_excel("dfnew.xlsx")
        dv=dx[dx.channal == "V:"]
        ########## connect the name of Measurement
        dv.index.name=x.strip('.txt')
        dx.index.name = x.strip('.txt')
        df_new.index.name=x.strip('.txt')

        dx=dx.reset_index()
        dv=dv.reset_index()
        df_new=df_new.reset_index()

        if not (dx.empty and deltas.empty):
            a[x.strip('.txt')]=dx
            lstdeltas[x.strip('.txt')]=df_new
            
        dv=dv.replace('V:', 'V:{}'.format(x.strip('.txt')))
        rows=df_new.shape[0]
        #print(rows)
        for row in range(rows):
            df_new["name"]=x.strip('.txt')
        #drop empty Dataframes
        if not (dx.empty):
            result=pd.concat([result,dx.append(pd.Series(name='V_Spacer',dtype='float'))],axis=1)
            verhaeltnis=pd.concat([verhaeltnis,dv],axis=1)
            new_df=result.append(verhaeltnis)
            deltas=pd.concat([deltas,df_new],axis=1)
            lstverhalt[x.strip('.txt')]=dv

        rows= deltas.shape[0]


#new_df=pd.concat([result,verhaeltnis])
#print(result.head())
#result.to_excel("result.xlsx")
verhaeltnis=verhaeltnis.drop(["max","deltas"], axis=1)
verhaeltnis=verhaeltnis.rename(columns={"min": "V"})
#verhaeltnis.head()
#verhaeltnis.to_excel("verhaeltnis.xlsx")
# verhaeltnis.to_excel("FullData.xlsx",sheet_name='verhältnis')
#deltas.to_excel("deltas.xlsx")
# deltas.to_excel("FullData.xlsx",sheet_name='deltas')
# new_df.to_excel("FullData.xls",sheet_name='fullData')
# with pd.ExcelWriter('FullDataAll.xlsx') as writer:
#     result.to_excel(writer, sheet_name='cleanValues')  
#     new_df.to_excel(writer, sheet_name='cleanWithVerhältnis')
#     deltas.to_excel(writer, sheet_name='deltas')
#     verhaeltnis.to_excel(writer, sheet_name='verhaeltnis')
#     new_df["spacer1"]=" "
#     new_df["spacer2"]=" "
#     merged_df=pd.merge(new_df,deltas.reset_index(),how='outer')
#     merged_df.to_excel(writer, sheet_name='FullDataAll')


In [10]:
#print(new_df.head())
#print(deltas.head())
new_df["spacer1"]=" "
new_df["spacer2"]=" "
merged_df=pd.merge(new_df,deltas.reset_index(),how='outer')
merged_df.to_excel("merged.xlsx")
new_df.tail()

,fischer1_p33_6_ch7_4_steel_brush,channal,min,max,deltas,fischer1_p33_6_hf8_fc6x_0_9km,channal,min,max,deltas,...,min,max,deltas,madshus_base7_ts_blue_poin,channal,min,max,deltas,spacer1,spacer2
5,59.0,V:fischer1_p33_6_ch7_4_steel_brush,1588.0,NaN,NaN,59.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1483.0,NaN,NaN,...,1058.0,NaN,NaN,59.0,V:madshus_base7_ts_blue_poin,1029.0,NaN,NaN,,
6,69.0,V:fischer1_p33_6_ch7_4_steel_brush,1406.0,NaN,NaN,69.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1734.0,NaN,NaN,...,393.0,NaN,NaN,69.0,V:madshus_base7_ts_blue_poin,1043.0,NaN,NaN,,
7,79.0,V:fischer1_p33_6_ch7_4_steel_brush,1226.0,NaN,NaN,79.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1547.0,NaN,NaN,...,550.0,NaN,NaN,79.0,V:madshus_base7_ts_blue_poin,1037.0,NaN,NaN,,
8,89.0,V:fischer1_p33_6_ch7_4_steel_brush,1320.0,NaN,NaN,89.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1735.0,NaN,NaN,...,865.0,NaN,NaN,89.0,V:madshus_base7_ts_blue_poin,1064.0,NaN,NaN,,
9,99.0,V:fischer1_p33_6_ch7_4_steel_brush,1598.0,NaN,NaN,99.0,V:fischer1_p33_6_hf8_fc6x_0_9km,1549.0,NaN,NaN,...,NaN,NaN,NaN,99.0,V:madshus_base7_ts_blue_poin,1004.0,NaN,NaN,,


In [29]:
dleta_names_list=list();
for x in lst:
    #print(x)
    dleta_names_list.append(x)
print(dleta_names_list)

['delta0', 'delta1', 'delta2', 'delta3', 'delta4', 'delta5', 'delta6', 'delta7', 'delta8']


In [52]:
if "delta5" in dleta_names_list:
    dleta_names_list.remove('delta5')
for x in lstdeltas:
    #print(lstdeltas[x].head())
    #fig, ax = plt.subplots()
    #ax.violinplot(lstdeltas[x][dleta_names_list].to_numpy(),bw_method=0.2);
    #sb.violinplot(data=lstdeltas[x][dleta_names_list[0:3]], inner="quartile", bw=0.2); 
    #lstdeltas[x][dleta_names_list].plot.box()
    # sb.violinplot(data=lstdeltas[x][dleta_names_list[0]], inner=None);
    # sb.swarmplot(data=lstdeltas[x][dleta_names_list[0]], size=2, color="k", alpha=0.5);
    #sb.boxplot(data=lstdeltas[x]);
    break;
    


















In [67]:
data_verhalt=pd.DataFrame();
delta_verhalt_name=list();
for x in lstverhalt:
    #print(lstverhalt[x]['min'])
    delta_verhalt_name.append(x)
    data_verhalt[x]=lstverhalt[x]['min'];
    # fig, ax = plt.subplots()
    # ax.violinplot(lstverhalt[x]['min'].to_numpy(),bw_method=0.2);
    #sb.violinplot(data=lstverhalt[x][dleta_names_list[0:3]], inner="quartile", bw=0.2); 
    #lstverhalt[x][dleta_names_list].plot.box()
    # sb.violinplot(data=lstverhalt[x][dleta_names_list[0]], inner=None);
    # sb.swarmplot(data=lstverhalt[x][dleta_names_list[0]], size=2, color="k", alpha=0.5);
print(delta_verhalt_name)




['fischer1_p33_6_ch7_4_steel_brush', 'fischer1_p33_6_hf8_fc6x_0_9km', 'fischer1_p33_6_hf8_fc6x_4km', 'fischer2_p33_6_ch7_4_steel_brush', 'fischer2_p33_6_hf8_fc6', 'fischer2_p33_6_hf8_fc6x_0_9km', 'fischer2_p33_6_hf8_fc6x_4km', 'fischer_bb1_TS_blue ch8 12km', 'fischer_bb1_TS_blue ch8', 'fischer_bb1_TS_blue remeasure 2', 'fischer_bb1_TS_blue remeasure', 'fischer_bb2_TS_blue LF8 12km', 'fischer_bb2_TS_blue LF8', 'fischer_bb2_TS_blue remeasure', 'fischer_p33_6_hf8_fc6', 'madshus_base7_ts_blue', 'madshus_base7_ts_blue_ clockwise 45 turn', 'madshus_base7_ts_blue_poin']


In [54]:
for x in verhaeltnis:
    print(x)

fischer1_p33_6_ch7_4_steel_brush
channal
V
fischer1_p33_6_hf8_fc6x_0_9km
channal
V
fischer1_p33_6_hf8_fc6x_4km
channal
V
fischer2_p33_6_ch7_4_steel_brush
channal
V
fischer2_p33_6_hf8_fc6
channal
V
fischer2_p33_6_hf8_fc6x_0_9km
channal
V
fischer2_p33_6_hf8_fc6x_4km
channal
V
fischer_bb1_TS_blue ch8 12km
channal
V
fischer_bb1_TS_blue ch8
channal
V
fischer_bb1_TS_blue remeasure 2
channal
V
fischer_bb1_TS_blue remeasure
channal
V
fischer_bb2_TS_blue LF8 12km
channal
V
fischer_bb2_TS_blue LF8
channal
V
fischer_bb2_TS_blue remeasure
channal
V
fischer_p33_6_hf8_fc6
channal
V
madshus_base7_ts_blue
channal
V
madshus_base7_ts_blue_ clockwise 45 turn
channal
V
madshus_base7_ts_blue_poin
channal
V
